In [ ]:
%%capture
!pip install datasets transformers
!pip install sentencepiece

In [ ]:
import datasets
import transformers
import pandas as pd
import numpy as np
import torch
from IPython.display import display, HTML
from transformers import pipeline

In [ ]:
from transformers import pipeline
# Load Trained Model
hub_model_id = "Alred/bart-base-finetuned-summarization-cnn-ver1.3"
summarizer = pipeline("summarization", model=hub_model_id, num_beams=6, no_repeat_ngram_size=3, temperature=2., do_sample=True)

In [ ]:
summarizer.model.config

BartConfig {
  "_name_or_path": "Alred/bart-base-finetuned-summarization-cnn-ver1.3",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2


In [ ]:
def preprocess_content(content, max_length=1024):
    input = summarizer.tokenizer(content, max_length=max_length, truncation=True)['input_ids']
    return summarizer.tokenizer.decode(input, skip_special_tokens=True)

In [ ]:
def run_sum_text(summarizer):
    """
    Subset Version
    """

    # Load Data 
    df_new_data = pd.read_csv("../Data/recent_news/bbc_news_updated.csv", index_col=0)
    df_new_data = df_new_data.iloc[:7, :]

    # Preprocess Input Text if needed
    df_new_data['Cleaned_Content'] = df_new_data["Content"].map(lambda x: preprocess_content(x))
    
    # Perform Summarization
    review = df_new_data["Cleaned_Content"].tolist()
    summary = summarizer(review)

    # Postprocess generated summaries
    cleaned_summary = list(map(lambda x: x['summary_text'], summary))
    output_df = pd.DataFrame({"Text": df_new_data['Content'], "Original Summary": df_new_data['Original Summary'], "Generated_summary": cleaned_summary})

    return output_df

In [ ]:
def run_sum_text_temp(temp_list):
    """
    Subset Version
    """

    col_name1 = ["summary_temp_" + str(temp) for temp in temp_list]
    col_name2 = []

    hub_model_id = "Alred/bart-base-finetuned-summarization-cnn-ver1.3"

    # Load Data 
    df_new_data = pd.read_csv("../Data/recent_news/bbc_news_updated.csv", index_col=0)
    df_new_data = df_new_data.iloc[:7, :]

    # Preprocess Input Text if needed
    df_new_data['Cleaned_Content'] = df_new_data["Content"].map(lambda x: preprocess_content(x))

    output_df = pd.DataFrame({"Text": df_new_data['Content'], "Original Summary": df_new_data['Original Summary']})

    for idx, temp in enumerate(temp_list):
        summarizer = pipeline("summarization", model=hub_model_id, no_repeat_ngram_size=3, temperature=temp, do_sample=True)
    
        # Perform Summarization
        review = df_new_data["Cleaned_Content"].tolist()
        summary = summarizer(review)

        # Postprocess generated summaries
        cleaned_summary = list(map(lambda x: x['summary_text'], summary))
        output_df[col_name1[idx]] = cleaned_summary

    return output_df

In [ ]:
def display_outcome(df, n):
    display(HTML(df.head(n).to_html()))

In [ ]:
output_df = run_sum_text(summarizer)

In [ ]:
output_df = run_sum_text_temp([1., 1.5, 2., 3.])

In [ ]:
display_outcome(output_df, 7)

In [ ]:
output_df.to_csv("temperature.csv")

In [ ]:
display_outcome(output_df, 7)